In [1]:
import numpy as np
import pandas as pd
import math
from scipy.optimize import minimize

In [17]:
df = pd.read_excel("market_data.xlsx", sheetname='Swaptions data')
df.head()

Market Volatilities for strike spreads in bps:  \
Tenor Expiry Fwd                                                         -150.0   
2     0.25   0.0107638332259373                                             0.0   
      0.5    0.011099189328091                                              0.0   
      0.75   0.0116024287527238                                             0.0   
      1      0.0121935636676584                                             0.0   

                                 Unnamed: 1  Unnamed: 2  Unnamed: 3  \
Tenor Expiry Fwd                  -100.0000    -50.0000    -25.0000   
2     0.25   0.0107638332259373      1.0470      0.4812      0.4327   
      0.5    0.011099189328091       0.9647      0.5079      0.4637   
      0.75   0.0116024287527238      0.8253      0.5033      0.4648   
      1      0.0121935636676584      0.6796      0.4788      0.4474   

                                 Unnamed: 4  Unnamed: 5  Unnamed: 6  \
Tenor Expiry Fwd                     0.0000     25.0000     50.0000   
2     0.25   0.0107638332259373      0.4268      0.4148      0.4253   
      0.5    0.011099189328091       0.4477      0.4390      0.4377   
      0.75   0.0116024287527238      0.4494      0.4387      0.4348   
      1      0.0121935636676584      0.4501      0.4435      0.4478   

                                 Unnamed: 7  Unnamed: 8  
Tenor Expiry Fwd                   100.0000    150.0000  
2     0.25   0.0107638332259373      0.4322      0.4495  
      0.5    0.011099189328091       0.4452      0.4576  
      0.75   0.0116024287527238      0.4375      0.4463  
      1      0.0121935636676584      0.4611      0.4754

In [68]:
column_names = df.loc['Tenor', : ].astype(str).values.tolist()
df.columns = column_names
df.index.name = ['Tenor','Expiry']
df = df.drop('Tenor')
df.head()

/home/mohilbajaj2002/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  after removing the cwd from sys.path.


-150.0  -100.0   -50.0   -25.0     0.0    25.0    50.0  \
2 0.25 0.010764     0.0  1.0470  0.4812  0.4327  0.4268  0.4148  0.4253   
  0.5  0.011099     0.0  0.9647  0.5079  0.4637  0.4477  0.4390  0.4377   
  0.75 0.011602     0.0  0.8253  0.5033  0.4648  0.4494  0.4387  0.4348   
  1    0.012194     0.0  0.6796  0.4788  0.4474  0.4501  0.4435  0.4478   
  2    0.016196     0.0  0.9119  0.5417  0.4628  0.4529  0.4461  0.4386   

                  100.0   150.0  
2 0.25 0.010764  0.4322  0.4495  
  0.5  0.011099  0.4452  0.4576  
  0.75 0.011602  0.4375  0.4463  
  1    0.012194  0.4611  0.4754  
  2    0.016196  0.4387  0.4442

In [84]:
# Extracting swaption characteristic values

column_names = pd.DataFrame(column_names)
strike_spreads = []
strike_spreads = column_names.apply(lambda x: x.astype(float)).values
num_strikes = len(strike_spreads)

index_values = []
index_values = df.index.values.tolist()

expiries = []
tenors = []
F = []

for i in range(1,len(index_values)):
    expiries.append(index_values[i][1])
    tenors.append(index_values[i][0])
    F.append(index_values[i][2])
    
# to create the strike grid
K = np.zeros([len(F),num_strikes], dtype = float)
for i in range(len(F)):
    for j in range(num_strikes):
        K[i][j] = F[i] + 0.0001*(strike_spreads[0][j])  

# to create market volatilities            
MKT = df.values


# set starting parameters
starting_guess = np.array([0.005,0.25,0,0.001])
alpha = len(F)*[starting_guess[0]]
beta = len(F)*[starting_guess[1]]
rho = len(F)*[starting_guess[2]]
nu = len(F)*[starting_guess[3]]

In [101]:
outvol = open('outvol.csv', 'w')             # file output of volatilities
vol_diff = open('vol differences.csv', 'w')  # file output differences between SABR and Market volatilities
parameters = open('parameters.csv', 'w') 

In [87]:
exp_dates = len(expiries)*[0]
for i in range(len(expiries)):
    if expiries[i] < 1:
        exp_dates[i] = str(int(round(12*expiries[i])))+'m'
    else:
        exp_dates[i] = str(int(round(expiries[i])))+'y'
        if expiries[i]-round(expiries[i]) > 0:
            exp_dates[i] = exp_dates[i]+str(int(round((12*(round(expiries[i],2)-int(expiries[i]))))))+'m' 
        elif expiries[i]-round(expiries[i]) < 0:
            exp_dates[i] = str(int(round(tenors[i]))-1)+'y'
            exp_dates[i] = exp_dates[i]+str(int(round((12*(round(expiries[i],2)-int(expiries[i]))))))+'m'

ten_dates = len(tenors)*[0]
for i in range(len(tenors)):
    if tenors[i] < 1:
        ten_dates[i] = str(int(round(12*tenors[i])))+'m'
    else:
        ten_dates[i] = str(int(round(tenors[i])))+'y'
        if tenors[i]-round(tenors[i]) > 0:
            ten_dates[i] = ten_dates[i]+str(int(round((12*(round(tenors[i],2)-int(tenors[i]))))))+'m' 
        elif tenors[i]-round(tenors[i]) < 0:
            ten_dates[i] = str(int(round(tenors[i]))-1)+'y'
            ten_dates[i] = ten_dates[i]+str(int(round((12*(round(tenors[i],2)-int(tenors[i]))))))+'m'

label_exp = exp_dates
label_ten = ten_dates
label_strikes = num_strikes*[0]
for i in range(num_strikes):
    if strike_spreads[0][i] == 0:
        label_strikes[i] = 'ATM'
    else:
        label_strikes[i] = str(strike_spreads[i])

In [102]:
def SABR(alpha,beta,rho,nu,F,K,time,MKT): # all variables are scalars

    if K <= 0:   # negative rates' problem, need to shift the smile
        VOL = 0
        diff = 0
    elif F == K: # ATM formula
        V = (F*K)**((1-beta)/2.)
        logFK = math.log(F/K)
        A = 1 + ( ((1-beta)**2*alpha**2)/(24.*(V**2)) + (alpha*beta*nu*rho)/(4.*V) + ((nu**2)*(2-3*(rho**2))/24.) ) * time
        B = 1 + (1/24.)*(((1-beta)*logFK)**2) + (1/1920.)*(((1-beta)*logFK)**4)
        VOL = (alpha/V)*A
        diff = VOL - MKT
    elif F != K: # not-ATM formula
        V = (F*K)**((1-beta)/2.)
        logFK = math.log(F/K)
        z = (nu/alpha)*V*logFK
        x = math.log( ( math.sqrt(1-2*rho*z+z**2) + z - rho ) / (1-rho) )
        A = 1 + ( ((1-beta)**2*alpha**2)/(24.*(V**2)) + (alpha*beta*nu*rho)/(4.*V) + ((nu**2)*(2-3*(rho**2))/24.) ) * time
        B = 1 + (1/24.)*(((1-beta)*logFK)**2) + (1/1920.)*(((1-beta)*logFK)**4)
        VOL = (nu*logFK*A)/(x*B)
        diff = VOL - MKT

    print (round(VOL,4) ,  '\t')
    outvol.write('%r;' %round(VOL,4) )
    if MKT==0:
        diff = 0
        vol_diff.write('%s;' %'No market data')
    else:
        vol_diff.write('%r;' %round(diff,4) )

In [103]:
def smile(alpha,beta,rho,nu,F,K,time,MKT,i): # F, time and the parameters are scalars, K and MKT are vectors, i is the index for tenor/expiry label

    print (label_ten[i] , '\t' , label_exp[i] , '\t')
    outvol.write('%s;%s;' %(label_ten[i],label_exp[i]))
    vol_diff.write('%s;%s;' %(label_ten[i],label_exp[i]))
    parameters.write('%s;%s;' %(label_ten[i],label_exp[i]))

    for j in range(len(K)):
        if K[0] <= 0:
            shift(F,K)
        SABR(alpha,beta,rho,nu,F,K[j],time,MKT[j])

    print (' ')
    outvol.write('\n')
    vol_diff.write('\n')
    parameters.write('%f;%f;%f;%f;' %(alpha ,beta ,rho ,nu))
    parameters.write('\n')

In [104]:
def SABR_vol_matrix(alpha,beta,rho,nu,F,K,time,MKT): # F, time and the parameters are vectors, K and MKT are matrices

    print (' ')
    print ((2+((num_strikes-1)/2)),'       ','SABR VOLATILITIES')
    print ('  ' , '\t' , 'strikes:') 
    for i in range(num_strikes):
        print (label_strikes[i] , '\t')
    print (' ')
    outvol.write('%s;' %'SABR VOLATILITIES')
    outvol.write('\n')
    vol_diff.write('%s;' %'VOLATILITY DIFFERENCES')
    vol_diff.write('\n')
    parameters.write('%s;' %'PARAMETERS')
    parameters.write('\n')
    outvol.write('%s;%s;' %(' ','strikes:'))
    vol_diff.write('%s;%s;' %(' ','strikes:'))
    for j in range(len(strike_spreads)):
        outvol.write('%s;' %label_strikes[j])
        vol_diff.write('%s;' %label_strikes[j])
    outvol.write('\n')
    vol_diff.write('\n')
    print ('tenor' , '\t' ,   'expiry')
    parameters.write('%s;%s;%s;%s;%s;%s' %('tenor','expiry','alpha','beta','rho','nu'))
    parameters.write('\n')

    for i in range(len(F)):
        smile(alpha[i],beta[i],rho[i],nu[i],F[i],K[i],time[i],MKT[i],i)

In [91]:
def forward_zero(F,K):
    shift = 0.001 - K[0]
    for j in range(len(K)):
        K[j] = K[j] + shift
        F = F + shift   

In [92]:
def obj_min(par,F,K,time,MKT):
    sum_sq_diff = 0
    if K[0]<=0:
        shift(F,K)
    for j in range(len(K)):
        if MKT[j] == 0:   
            diff = 0       
        elif F == K[j]: 
            V = (F*K[j])**((1-par[1])/2.)
            logFK = math.log(F/K[j])
            A = 1 + ( ((1-par[1])**2*par[0]**2)/(24.*(V**2)) + (par[0]*par[1]*par[3]*par[2])/(4.*V) + ((par[3]**2)*(2-3*(par[2]**2))/24.) ) * time
            B = 1 + (1/24.)*(((1-par[1])*logFK)**2) + (1/1920.)*(((1-par[1])*logFK)**4)
            VOL = (par[0]/V)*A
            diff = VOL - MKT[j]
        elif F != K[j]: 
            V = (F*K[j])**((1-par[1])/2.)
            logFK = math.log(F/K[j])
            z = (par[3]/par[0])*V*logFK
            x = math.log( ( math.sqrt(1-2*par[2]*z+z**2) + z - par[2] ) / (1-par[2]) )
            A = 1 + ( ((1-par[1])**2*par[0]**2)/(24.*(V**2)) + (par[0]*par[1]*par[3]*par[2])/(4.*V) + ((par[3]**2)*(2-3*(par[2]**2))/24.) ) * time
            B = 1 + (1/24.)*(((1-par[1])*logFK)**2) + (1/1920.)*(((1-par[1])*logFK)**4)
            VOL = (par[3]*logFK*A)/(x*B)
            diff = VOL - MKT[j]  
        sum_sq_diff = sum_sq_diff + diff**2  
        obj = math.sqrt(sum_sq_diff)
    return obj

In [93]:
def calibration(starting_par,F,K,time,MKT):
    for i in range(len(F)):
        x0 = starting_par
        bnds = ( (0.001,None) , (0,1) , (-0.999,0.999) , (0.001,None)  )
        res = minimize(obj_min, x0 , (F[i],K[i],time[i],MKT[i]) ,bounds = bnds, method='SLSQP') # for a constrained minimization of multivariate scalar functions
        alpha[i] = res.x[0]
        beta[i] = res.x[1]
        rho[i] = res.x[2]
        nu[i] = res.x[3]

In [105]:
calibration(starting_guess,F,K,expiries,MKT)

In [106]:
SABR_vol_matrix(alpha,beta,rho,nu,F,K,expiries,MKT)

 
2.0         SABR VOLATILITIES
   	 strikes:
[-150. -100.  -50.  -25.    0.   25.   50.  100.  150.] 	
 
tenor 	 expiry
2y 	 6m 	
0.3164 	
 
2y 	 9m 	
0.3099 	
 
2y 	 1y 	
0.3027 	
 
2y 	 2y 	
0.2518 	
 
2y 	 5y 	
0.0945 	
 
2y 	 10y 	
0.404 	
 
5y 	 3m 	
0.3026 	
 
5y 	 6m 	
1.187 	
 
5y 	 9m 	
0.9568 	
 
5y 	 1y 	
0.8325 	
 
5y 	 2y 	
0.7242 	
 
5y 	 5y 	
0.5704 	
 
5y 	 10y 	
0.372 	
 
10y 	 3m 	
0.3108 	
 
10y 	 6m 	
0.6333 	
 
10y 	 9m 	
0.6149 	
 
10y 	 1y 	
0.586 	
 
10y 	 2y 	
0.5544 	
 
10y 	 5y 	
0.4725 	
 
10y 	 10y 	
0.3598 	
 
15y 	 3m 	
0.3363 	
 
15y 	 6m 	
0.5802 	
 
15y 	 9m 	
0.5492 	
 
15y 	 1y 	
0.5122 	
 
15y 	 2y 	
0.4938 	
 
15y 	 5y 	
0.4582 	
 
15y 	 10y 	
0.3815 	
 
30y 	 3m 	
0.3545 	
 
30y 	 6m 	
0.628 	
 
30y 	 9m 	
0.5954 	
 
30y 	 1y 	
0.5678 	
 
30y 	 2y 	
0.5536 	
 
30y 	 5y 	
0.5056 	
 
30y 	 10y 	
0.4261 	
 
3y9m 	 2y2m 	
0.3664 	
 


In [107]:
outvol.close()
vol_diff.close()
parameters.close()